In [ ]:
import tensorflow as tf
import os 
import zipfile
import pandas as pd
import numpy as np

# **Sarcasm Detector using NLP**

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving kaggle.json to kaggle (1).json


In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

Processing /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674/kaggle-1.5.6-cp36-none-any.whl
  Found existing installation: kaggle 1.5.6
    Uninstalling kaggle-1.5.6:
      Successfully uninstalled kaggle-1.5.6


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


#### Importing kaggle file

In [ ]:
!kaggle datasets download -d danofer/sarcasm -p "/tmp"

sarcasm.zip: Skipping, found more recently modified local copy (use --force to force download)


#### Extract File

In [ ]:
local_zip='/tmp/sarcasm.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
ds_csv=pd.read_csv("/tmp/train-balanced-sarcasm.csv")

In [ ]:
ds_csv.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [ ]:
ds_csv.shape

(1010826, 10)

### Dropping un-important column

In [ ]:
ds_csv=ds_csv.drop(columns=['author','subreddit',"score","ups","downs","date","created_utc","parent_comment"],axis=1)

In [ ]:
ds_csv.head()

,label,comment
0,0,NC and NH.
1,0,You do know west teams play against west teams...
2,0,"They were underdogs earlier today, but since G..."
3,0,"This meme isn't funny none of the ""new york ni..."
4,0,I could use one of those tools.


#### Shuffle the dataset

In [ ]:
ds_csv=ds_csv.sample(frac=1) #generate random rows with frac=fraction*length of df

In [ ]:
ds_csv.shape

(1010826, 2)

In [ ]:
train=ds_csv.iloc[:50000,:]
val=ds_csv.iloc[50000:51000,:]

In [ ]:
train.shape # training Set


(10000, 2)

In [ ]:
val.shape #Validation Set

(1000, 2)

In [ ]:
val.head()

,label,comment
622390,1,"You know, That outta work!"
9545,0,Which they said was a rip off of an encycloped...
713530,0,Guess you haven't heard of multisignature escr...
521607,1,"Hey, but it got Dri Archer into that game!"
245099,0,Apparently Waite's recovered quite quickly and...


###**List of String :: labels = numpy array**

In [ ]:
train_sentences=[]
val_sentences=[]

tr_sen_list=train['comment'].tolist()
#tr_lb_list=train['label'].tolist()
vl_sen_list=val['comment'].tolist()
#vl_lb_list=val['label'].tolist()

for s in tr_sen_list:
  train_sentences.append(str(s)) #Type cast to str to avoid error

for s in vl_sen_list:
  val_sentences.append(str(s)) #Type cast to str to avoid error

train_label=ds_csv.iloc[:50000,0].values
val_label=ds_csv.iloc[50000:51000,0].values

In [ ]:
len(val_label)

10826

## **Starting preprocessing on sentences**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#defining Hyper-parameter
vocab_size=5000
max_length=90
trunc="post"
pad="post"
unk_tkn="<unk>"
embedding_dim=32

tokenizer=Tokenizer(num_words=vocab_size,oov_token=unk_tkn)
tokenizer.fit_on_texts(train_sentences)
word_index=tokenizer.word_index

#Sequencing and padding on training set
train_sequences=tokenizer.texts_to_sequences(train_sentences)
train_padded=pad_sequences(train_sequences,padding=pad,maxlen=max_length,truncating=trunc)

#Sequencing and padding on validation set
val_sequences=tokenizer.texts_to_sequences(val_sentences)
val_padded=pad_sequences(val_sequences,padding=pad,maxlen=max_length,truncating=trunc)



In [ ]:
train_padded

array([[ 384,   26,    7, ...,    0,    0,    0],
       [  48,   31,   19, ...,    0,    0,    0],
       [   2, 1120,  101, ...,    0,    0,    0],
       ...,
       [  27,    7, 1198, ...,    0,    0,    0],
       [  35,   87, 1636, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int32)

##**Applying Neural Netwok**

In [ ]:
model=tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
      #tf.keras.layers.Conv1D(128, 5, activation="relu"),
      #tf.keras.layers.MaxPooling1D(2,padding="same"),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
      #tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
      tf.keras.layers.Dense(64,activation='relu'),
      tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='RMSprop',metrics=['accuracy'])

In [ ]:
num_epochs=5
model.fit(train_padded,train_label,epochs=num_epochs,
          validation_data=(val_padded,val_label))

Epoch 1/5
1563/1563 [==============================] - 63s 40ms/step - loss: 0.6172 - accuracy: 0.6557 - val_loss: 0.6023 - val_accuracy: 0.6780
Epoch 2/5
1563/1563 [==============================] - 63s 40ms/step - loss: 0.5723 - accuracy: 0.7015 - val_loss: 0.5839 - val_accuracy: 0.6910
Epoch 3/5
1563/1563 [==============================] - 62s 40ms/step - loss: 0.5545 - accuracy: 0.7160 - val_loss: 0.5864 - val_accuracy: 0.7020
Epoch 4/5
1563/1563 [==============================] - 62s 40ms/step - loss: 0.5409 - accuracy: 0.7269 - val_loss: 0.6017 - val_accuracy: 0.6920
Epoch 5/5
1563/1563 [==============================] - 63s 40ms/step - loss: 0.5285 - accuracy: 0.7345 - val_loss: 0.5945 - val_accuracy: 0.6890


## Using SVM classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
classifier=SVC(kernel='linear',random_state=42)
classifier.fit(train_padded,train_label)


In [ ]:
val_pred=classifier.predict(val_padded)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(val_label,val_pred)

array([[103, 400],
       [ 94, 403]])